
# Preprocessing des docs

In [1]:
import os
import nltk
from nltk import WordNetLemmatizer
import json
import re
import collections

def read_documents(json_file_path):
    """
    Lit le fichier JSON, extrait les ids et les chemins des fichiers, lit le contenu de chaque fichier,
    et retourne un dictionnaire associant chaque id à son contenu.

    :param json_file_path: Chemin du fichier JSON contenant les métadonnées.
    :return: Un dictionnaire {id: contenu}.
    """
    try:
        # Lire le fichier JSON
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            documents = json.load(json_file)
        
        # Initialiser le dictionnaire pour les résultats
        doc_contents = {}

        # Parcourir les documents
        for doc in documents:
            doc_id = doc.get("id")
            file_path = doc.get("file_path")
            
            # Vérifier si le chemin du fichier est valide
            if doc_id and file_path and os.path.exists(file_path):
                # Lire le contenu du fichier
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                doc_contents[doc_id] = content
            else:
                print(f"Fichier introuvable ou informations manquantes pour l'id {doc_id}: {file_path}")
        
        return doc_contents

    except Exception as e:
        print(f"Une erreur s'est produite: {e}")
        return {}


def pretraitement(document_content, vocabulary):
    """
    Prend en entrée le contenu des fichiers et applique le prétraitement :
    - Tokenisation
    - Suppression des stopwords
    - Conversion en minuscules
    - Lemmatisation
    - Stemming
    """
    stopwords = nltk.corpus.stopwords.words('french')
    
    # Instanciation des objets WordNetLemmatizer 
    lemmatizer = WordNetLemmatizer()
    
    vocabulary = collections.Counter()
    for file, file_content in documents_content.items():
        file_content = re.sub('[0-9]+',' ', file_content)
        # Tokenisation des mots dans le contenu du fichier
        tokens = nltk.regexp_tokenize(file_content, pattern=r'\w+')  # Tokenisation avec le bon pattern
        
        # Suppression des stopwords et conversion en minuscules
        tokens = [token.lower() for token in tokens if token.lower() not in stopwords]
        
        # Lemmatisation
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Mise à jour du contenu du fichier traité dans le dictionnaire
        documents_content[file] = tokens
        
        # Ajout des mots au vocabulaire global (flatten la liste)
        vocabulary.update(tokens)
        # Sauvegarder dans un fichier JSON
    data_to_save = {
        "preprocessed_content": documents_content,
        "vocabulary": list(vocabulary.keys())
    }
        
    with open('preprocessed_content.json', 'w', encoding='utf-8') as json_file:
        json.dump(data_to_save, json_file, indent=4)
    print(f"Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier :preprocessed_content.json.")

            

# Application

json_file_path = "all_documents.json"
documents_content = read_documents(json_file_path)
vocabulary = []
pretraitement(documents_content, vocabulary)


Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier :preprocessed_content.json.


# Representation des documents sous forme de vecteur

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import json

def load_preprocessed_content(input_file="preprocessed_content.json"):
    """
    Charge le contenu prétraité et le vocabulaire depuis un fichier JSON.

    :param input_file: Nom du fichier JSON contenant les données.
    :return: Tuple (preprocessed_content, vocabulary).
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        
        preprocessed_content = data.get("preprocessed_content", {})
        vocabulary = data.get("vocabulary", [])
        return preprocessed_content, vocabulary
    except FileNotFoundError:
        print(f"Erreur : Le fichier {input_file} n'a pas été trouvé.")
        return {}, []
    except json.JSONDecodeError:
        print(f"Erreur : Le fichier {input_file} n'est pas un fichier JSON valide.")
        return {}, []
    except Exception as e:
        print(f"Une erreur s'est produite lors de la lecture du fichier : {e}")
        return {}, []

def create_doc2vec_model(doc_contents, output_file="document_vectors.json", vector_size=200, epochs=20):
    """
    Crée un modèle Doc2Vec, calcule le vecteur de chaque document et enregistre les vecteurs dans un fichier JSON.

    :param doc_contents: Dictionnaire {id: contenu du document}.
    :param output_file: Nom du fichier pour enregistrer les vecteurs.
    :param vector_size: Taille des vecteurs générés.
    :param epochs: Nombre d'époques pour entraîner le modèle.
    """
    try:
        # Vérification si doc_contents est vide
        if not doc_contents:
            print("Erreur : Aucun contenu de document fourni pour entraîner le modèle.")
            return
        
        # Préparer les données pour Doc2Vec
        tagged_documents = [
            TaggedDocument(words=content, tags=[doc_id]) 
            for doc_id, content in doc_contents.items()
        ]
        
        # Initialiser et entraîner le modèle
        model = Doc2Vec(vector_size=vector_size, min_count=1, epochs=epochs, workers=4)
        model.build_vocab(tagged_documents)
        model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)
        # Sauvegarder le modèle Doc2Vec
        model.save("doc2vec_model.model")
        print("Le modèle Doc2Vec a été sauvegardé sous le nom 'doc2vec_model.model'.")

        # Calculer les vecteurs des documents
        document_vectors = {doc_id: model.dv[doc_id].tolist() for doc_id in doc_contents.keys()}
        
        # Enregistrer les vecteurs dans un fichier JSON
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(document_vectors, json_file, indent=4)
        
        print(f"Les vecteurs des documents ont été enregistrés dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Application
if __name__ == "__main__":
    # Charger les contenus prétraités
    preprocessed_content, vocabulary = load_preprocessed_content()
    
    # Vérifier si des contenus ont été chargés avec succès
    if preprocessed_content:
        # Créer le modèle Doc2Vec et calculer les vecteurs
        create_doc2vec_model(preprocessed_content, output_file="document_vectors.json")
    else:
        print("Impossible de créer le modèle Doc2Vec car aucun contenu prétraité n'a été chargé.")


Le modèle Doc2Vec a été sauvegardé sous le nom 'doc2vec_model.model'.
Les vecteurs des documents ont été enregistrés dans le fichier document_vectors.json.


# Le calcule de similarites

In [3]:
import csv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

def compute_similarity_matrix(vector_file="document_vectors.json", output_file="document_similarity_matrix.csv"):
    """
    Calcule la similarité cosinus entre tous les documents et enregistre les résultats dans un fichier CSV sous forme de matrice.

    :param vector_file: Nom du fichier JSON contenant les vecteurs des documents.
    :param output_file: Nom du fichier CSV pour enregistrer la matrice des similarités.
    """
    try:
        # Charger les vecteurs depuis le fichier JSON
        with open(vector_file, 'r', encoding='utf-8') as json_file:
            document_vectors = json.load(json_file)
        
        # Extraire les IDs et les vecteurs
        doc_ids = list(document_vectors.keys())
        vectors = np.array([document_vectors[doc_id] for doc_id in doc_ids])
        
        # Calculer la matrice de similarité cosinus
        similarity_matrix = cosine_similarity(vectors)
        
        # Enregistrer la matrice dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (IDs des documents)
            writer.writerow([""] + doc_ids)
            
            # Écrire chaque ligne (ID + similarités)
            for i, doc_id in enumerate(doc_ids):
                writer.writerow([doc_id] + similarity_matrix[i].tolist())
        
        print(f"La matrice de similarité a été enregistrée dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
import csv

def save_sorted_similarities_from_matrix(matrix_file="document_similarity_matrix.csv", output_file="sorted_document_similarities.csv"):
    """
    Enregistre dans un fichier CSV les IDs des documents les plus similaires pour chaque document,
    triés par similarité décroissante, à partir d'une matrice de similarité déjà calculée.

    :param matrix_file: Nom du fichier CSV contenant la matrice de similarité.
    :param output_file: Nom du fichier CSV pour enregistrer les similarités triées (seulement les IDs).
    """
    try:
        # Charger la matrice de similarité depuis le fichier CSV
        with open(matrix_file, 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            rows = list(reader)
        
        # Extraire les IDs des documents (en-tête)
        doc_ids = rows[0][1:]
        
        # Préparer les similarités triées (seulement les IDs)
        sorted_similarities = []
        for i, row in enumerate(rows[1:]):
            doc_id = row[0]
            similarities = [(doc_ids[j], float(row[j + 1])) for j in range(len(doc_ids)) if i != j]
            
            # Trier les similarités par ordre décroissant et récupérer uniquement les IDs
            sorted_doc_ids = [sim_doc_id for sim_doc_id, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]
            sorted_similarities.append((doc_id, sorted_doc_ids))
        
        # Enregistrer les IDs triés dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (Document + Liste d'IDs des documents similaires)
            header = ["Document"] + [f"Similar Doc {i+1}" for i in range(len(doc_ids) - 1)]
            writer.writerow(header)
            
            # Écrire les similarités triées pour chaque document
            for doc_id, sorted_doc_ids in sorted_similarities:
                # Compléter avec des colonnes vides si moins de documents similaires
                row = [doc_id] + sorted_doc_ids + [''] * (len(doc_ids) - 1 - len(sorted_doc_ids))
                writer.writerow(row)
        
        print(f"Les similarités triées ont été enregistrées dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Application
if __name__ == "__main__":
    compute_similarity_matrix(vector_file="document_vectors.json", output_file="document_similarity_matrix.csv")
    save_sorted_similarities_from_matrix(
        matrix_file="document_similarity_matrix.csv",
        output_file="sorted_document_similarities.csv"
    )


La matrice de similarité a été enregistrée dans le fichier document_similarity_matrix.csv.
Les similarités triées ont été enregistrées dans le fichier sorted_document_similarities.csv.
